<h1><center>TP3 Introduction à la classification</center></h1>

# Plus Proche Voisin

## importer les bibliotheques

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import metrics
from collections import Counter
from functools import reduce
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from scipy.spatial import distance

## chargement des données

In [3]:
iris = datasets.load_iris()
features = iris.data
targets = iris.target

## Creation de la fonction PPV(X,Y)

In [4]:
def error(target_1, target_2):
#     size = len(target_1)
#     cmp = 0
#     for i in range(size):
#         if target_1[i] != target_2[i]:
#             cmp += 1
#     r = cmp*100/size
    return (1-(metrics.accuracy_score(target_1,target_2)))*100

def PPV(X,Y):
    #calculate data size
    data_size = len(X)
    #inisialize new targets with zeros
    new_target = np.zeros(data_size)
    for index in range(data_size):
        #select variable
        var = X[index]
        #delete variable from training data
        train = np.delete(X, index, 0)
        #delete variable classe name from training targets
        train_target = np.delete(Y, index, 0)
        #check if element was deleting
        if data_size-1 != len(train):exit()
        # calculate euclidean_distances betwen selected variable and training data
        dist = list(euclidean_distances(train,var.reshape(1,-1)))
        # Select class of the closest data
        min_dist_idx = np.argmin(dist)
        new_target[index] = train_target[min_dist_idx]       
    return {'target': new_target, 'error': error(new_target,Y)}   

PPV(features,targets)

{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 'error': 4.0000000000000036}

## Test KNN classifier de sklearn pour plusieur valuer de k 

In [57]:
def KNN(features,targets,k):
    data_size = len(features)
    #inisialize new targets with zeros
    new_target = np.zeros(data_size)
    for index in range(data_size):
        #select variable
        var = features[index]
        #delete variable from training data
        train = np.delete(features, index, 0)
        #delete variable classe name from training targets
        train_target = np.delete(targets, index, 0)
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(train, train_target)
        new_target[index] = model.predict(var.reshape(1,-1))
    return {'target': new_target, 'error': error(new_target,targets)} 

# # pour k = 1 
print(KNN(features,targets,1))
# # pour k = 5
print(KNN(features,targets,5))
# # pour k = 10 
print(KNN(features,targets,19))

{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]), 'error': 4.0000000000000036}
{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

## la fonction PPV après mise à jour

In [69]:
def majorety(arr):
    uniques = np.unique(arr)
    res = []
    for u in uniques:
        tmp = [u,0,0]
        cmp = 0
        for e,i in zip(arr,range(len(arr))):
            if e == u :
                tmp[1] +=1
                if cmp == 0:
                    tmp[2] = i
                    cmp += 1
        res.append(tmp)
    best = res[0]
    arr = list(np.delete(np.array(res),0,axis=0))
    for t in res:
        if best[1] < t[1]:
            best = t
        else:
            if best[1] == t[1]:
                if best[2] > t[2]:
                    best = t       
    return best[0]
            
arr = np.array([ 3, 3, 2, 2, 1, 1])
print(majorety(arr))
np.bincount(arr).argmax()

3


1

In [64]:
def PPV_1(X,Y,K):
    #calculate data size
    data_size = len(X)
    #inisialize new targets with zeros
    new_target = np.zeros(data_size)
    for index in range(data_size):
        #select variable
        var = X[index]
        #delete variable from training data
        train = np.delete(X, index, 0)
        #delete variable classe name from training targets
        train_target = np.delete(Y, index, 0)
        #check if element was deleting
        if data_size-1 != len(train):exit()
        # calculate euclidean_distances betwen selected variable and training data
        dist = list(euclidean_distances(train,var.reshape(1,-1)))
        distance = np.array(euclidean_distances(train,var.reshape(1,-1))).flatten()
        if K == 1:
            # Select class of the closest data
            min_dist_idx = np.argmin(dist)
            new_target[index] = train_target[min_dist_idx]
        else :
            #sort indexes of closed variables
            sorted_dist = distance.argsort()
            #get indexes of closed variables
            kmin_dist = sorted_dist[0:K]
            # get the index class for each elements in k closed variables
            kmin_dist_index = [ train_target[i] for i in kmin_dist ]
#             print(kmin_dist_index)
            #get the majority class
            index_max = majorety(kmin_dist_index)#np.bincount(kmin_dist_index).argmax()
#             print(kmin_dist_index)
#             print(index_max)
            new_target[index] = index_max
            
    return {'target': new_target, 'error': error(new_target,Y)}   

PPV_1(features,targets,5)

{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
        2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 'error': 3.3333333333333326}

## Classifieur Bayesien Naïf

## Creation de la fonction CBN(X,Y)

In [93]:
def CBN(X,Y):
    #number of classes
    nb_class = np.unique(Y).size
    # data size
    data_size = len(X)
    # number of features
    nb_features = X.shape[1]
    #print(nb_features)
    #print(Counter(Y).values())
    #print(proba_class)
    #inisialize new targets with zeros
    new_target = np.zeros(data_size)
    for index in range(data_size):
        #select variable
        var = X[index]
        #delete variable from training data
        train = np.delete(X, index, 0)
        #delete variable class from training targets
        train_target = np.delete(Y, index, 0)
        #check if element was deleting
        if data_size-1 != len(train):exit()
        #calculate the probability for eache classe
        proba_class = [e / float(len(train)) for e in Counter(train_target).values()]
        proba_class = np.array(proba_class).reshape(-1,1) 
        # Groups data by class and calculates their centers of gravity
        data_pclass = {}
        mean_pclass = {}
        for j in range(nb_class):
            #get data for eache class
            data_pclass[j] = [e for k, e in enumerate(train) if train_target[k] == j]
            #calculate the mean of variables for eache classe
            mean_pclass[j] = np.mean(a=data_pclass[j], axis=0)
        matrix = np.array([mean_pclass[i] for i in mean_pclass])
        # Calculate distances
        # dist represents the distance between the X-data and the center of gravity for each class
        # dist_total is the sum of the distances by class              
        dist = euclidean_distances(var.reshape(1,-1),matrix)
        dist_total = np.sum(dist)
        # Calculates the probability PROD(P(xi/wk)P(wk))
        # xi/wk = une donne x avec la valeur xi pour la variable i de la classe wk
        res = 1 - dist / dist_total #np.divide(1 - dist, dist_total)
        pred = np.prod( res * proba_class , axis = 0)
        new_target[index] = pred.argmax()
#         print(matrix)
#         print(var)
#         print(dist)
#         print(dist_total)
#         print(res)
#         print(proba_class)
#         print(res * proba_class)
#         print(pred)
#         print(pred.argmax())
    return {'target': new_target, 'error': error(new_target, Y)}
    
print(CBN(features,targets))

{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.,
       1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 2., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2.,
       1., 2., 1., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]), 'error': 7.9999999999999964}


## Test du fonction du Classifieur Bayesien Naïf inclut dans sklearn

In [94]:
def BNS(features,targets):
    data_size = len(features)
    #inisialize new targets with zeros
    new_target = np.zeros(data_size)
    for index in range(data_size):
        #select variable
        var = features[index]
        #delete variable from training data
        train = np.delete(features, index, 0)
        #delete variable classe name from training targets
        train_target = np.delete(targets, index, 0)
        model = GaussianNB()
        model.fit(train, train_target)
        new_target[index] = model.predict(var.reshape(1,-1))
    return {'target': new_target, 'error': error(new_target,targets)}  
print(BNS(features,targets))

{'target': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]), 'error': 4.666666666666663}


In [88]:
import math
x = [5.006 ,3.428 ,1.462 ,0.246]
y = [7. ,3.2 ,4.7 ,1.4]
math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))

3.98049996859691